## Importation et modification données

In [ ]:
# Start writing code here...from sklearn.cluster import KMeans
import pandas as pd
import numpy as np
from sklearn.manifold import MDS
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial.distance import cdist
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [ ]:
spotify_songs = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-01-21/spotify_songs.csv')


Q1 = np.percentile(spotify_songs.duration_ms, 25,
                   interpolation = 'midpoint')
Q3 = np.percentile(spotify_songs.duration_ms, 75,
                   interpolation = 'midpoint')
IQR = Q3 - Q1

"""
mask = (spotify_songs.duration_ms < (Q3+1.5*IQR))
mask2 = spotify_songs.duration_ms > (Q1-1.5*IQR)
spotify_data_cleaned = spotify_songs.loc[mask]
spotify_data_cleaned = spotify_data_cleaned.loc[mask2]
"""
spotify_data_cleaned = spotify_songs
spotify_data_cleaned.track_album_release_date.str[:4]
spotify_data_cleaned.track_album_release_date.str[5:7]
spotify_data_cleaned = spotify_data_cleaned.assign(
    track_album_release_month = spotify_data_cleaned.track_album_release_date.str[5:7],
    track_album_release_year = spotify_data_cleaned.track_album_release_date.str[:4],
)
spotify_data_cleaned.drop(columns="track_album_release_date",inplace=True)
spotify_data_cleaned.track_album_release_year = spotify_data_cleaned.track_album_release_year.astype("int64")
"""
mask = (spotify_data_cleaned.track_popularity > 0)
spotify_data_cleaned = spotify_data_cleaned.loc[mask]
"""


'\nmask = (spotify_data_cleaned.track_popularity > 0)\nspotify_data_cleaned = spotify_data_cleaned.loc[mask]\n'

In [ ]:
from sklearn.preprocessing import MinMaxScaler

def normalisation(data):
    columns = ["track_popularity","danceability","energy","key","loudness","speechiness","acousticness","instrumentalness","liveness","valence","tempo", "duration_ms", "track_album_release_year"]
    data_norme = data[columns]
    scaler = MinMaxScaler(feature_range=(0,1))
    data_norme = pd.DataFrame(scaler.fit_transform(data_norme), columns=columns)
    return data_norme


## KNN

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

**Fonction pour appliquer la KNN**

In [ ]:
def KNN(data_normed_quanti,data):
    xx = data_normed_quanti
    yy = data.playlist_genre
    x_train, x_test, y_train, y_test = train_test_split(xx, yy, train_size=0.66)

    n_neighbors_list = np.unique(np.round(np.geomspace(1, 500, 100)).astype(int))
    param_grid = {"n_neighbors": n_neighbors_list}

    cls = KNeighborsClassifier()

    search = GridSearchCV(cls, param_grid, scoring="accuracy", cv=10)
    search.fit(xx, yy)
    kopt = search.best_params_
    print(kopt)
    
    knn = KNeighborsClassifier(n_neighbors = kopt["n_neighbors"])
    #training
    knn.fit(x_train,y_train)
    print(knn.score(x_test,y_test))

**Fonction pour appliquer KNN sur 2 genres**

In [ ]:
def KNN_2_genre(genre1,genre2):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2

    data_1= spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    
    data= pd.concat([data_1,data_2])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 3 genres**

In [ ]:
def KNN_3_genre(genre1,genre2,genre3):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3

    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]

    data= pd.concat([data_1,data_2,data_3])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Fonction pour appliquer KNN sur 4 genres**

In [ ]:
def KNN_4_genre(genre1,genre2,genre3,genre4):
    mask_1 = spotify_data_cleaned.playlist_genre == genre1
    mask_2 = spotify_data_cleaned.playlist_genre == genre2
    mask_3 = spotify_data_cleaned.playlist_genre == genre3
    mask_4 = spotify_data_cleaned.playlist_genre == genre4


    data_1 = spotify_data_cleaned.loc[mask_1]
    data_2 = spotify_data_cleaned.loc[mask_2]
    data_3 = spotify_data_cleaned.loc[mask_3]
    data_4 = spotify_data_cleaned.loc[mask_4]

    data= pd.concat([data_1,data_2,data_3,data_4])

    data_normed_quanti = normalisation(data)
    
    KNN(data_normed_quanti,data)

**Application sur tous les cas**

In [ ]:
print("EDM - RAP")
KNN_2_genre("edm","rap")
print("---------------------")

print("EDM - ROCK")
KNN_2_genre("edm","rock")
print("---------------------")

print("EDM - R&B")
KNN_2_genre("edm","r&b")
print("---------------------")

print("EDM - latin")
KNN_2_genre("edm","latin")
print("---------------------")

print("EDM - pop")
KNN_2_genre("edm","pop")
print("---------------------")

print("RAP - ROCK")
KNN_2_genre("rap","rock")
print("---------------------")

print("RAP - R&B")
KNN_2_genre("rap","r&b")
print("---------------------")

print("RAP - LATIN")
KNN_2_genre("rap","latin")
print("---------------------")

print("rap - pop")
KNN_2_genre("rap","pop")
print("---------------------")

print("rock - r&b")
KNN_2_genre("rock","r&b")
print("---------------------")

print("rock - latin")
KNN_2_genre("rock","latin")
print("---------------------")

print("rock - pop")
KNN_2_genre("rock","pop")
print("---------------------")

print("r&b - latin")
KNN_2_genre("r&b","latin")
print("---------------------")

print("r&b - pop")
KNN_2_genre("r&b","pop")
print("---------------------")

print("latin - pop")
KNN_2_genre("latin","pop")


EDM - RAP
{'n_neighbors': 13}
0.8336243452232477
---------------------
EDM - ROCK
{'n_neighbors': 9}
0.8879079721776351
---------------------
EDM - R&B
{'n_neighbors': 15}
0.8534085084572014
---------------------
EDM - latin
{'n_neighbors': 25}
0.7912289915966386
---------------------
EDM - pop
{'n_neighbors': 71}
0.7443340972752738
---------------------
RAP - ROCK
{'n_neighbors': 15}
0.9139400604894143
---------------------
RAP - R&B
{'n_neighbors': 25}
0.745067087608524
---------------------
RAP - LATIN
{'n_neighbors': 9}
0.7483140005395198
---------------------
rap - pop
{'n_neighbors': 9}
0.7836425398484452
---------------------
rock - r&b
{'n_neighbors': 18}
0.8453257790368272
---------------------
rock - latin
{'n_neighbors': 9}
0.8792551643875473
---------------------
rock - pop
{'n_neighbors': 15}
0.8200224971878515
---------------------
r&b - latin
{'n_neighbors': 17}
0.7275
---------------------
r&b - pop
{'n_neighbors': 23}
0.734068297929551
---------------------
latin - pop

- moyenne à 79,5%

**Application sur les trois genres qui semblent le plus facile à prédire**

In [ ]:
KNN_3_genre("edm",'rap','rock')

{'n_neighbors': 11}
0.8021784961349262


**Tentative sur 4 genres**


In [ ]:
KNN_4_genre("edm",'rap','rock','pop')

{'n_neighbors': 25}
0.6560021152829191


In [ ]:
KNN_4_genre("edm",'rap','rock','latin')

{'n_neighbors': 16}
0.6729348556077904


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=c2ad4461-7f35-42f5-b2d9-f686e8952f67' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>